## Install Dependencies

In [ ]:
!pip install langchain==0.3.4
!pip install langchain-openai==0.2.3
!pip install langchain-community==0.3.3
!pip install jq==1.8.0
!pip install pymupdf==1.24.12
!pip install httpx==0.27.2
# install vectordb and bm25 utils
!pip install langchain-chroma==0.1.4
!pip install rank_bm25==0.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.6
    Uninstalling langsmith-0.3.6:
      Successfully uninstalled langsmith-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 8.8 MB/s eta 0:00:00


## Setup Environment Variables

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "API_KEY"

## Get the Dataset


In [ ]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
docs[1]

Document(metadata={'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf', 'page': 1}, page_content=' \na) os detentores de mandato eletivo; \nb) os ocupantes de cargos vitalícios; \nc) os ocupantes de cargos de direção ou de chefia. \n§ 2º Pelo menos cinquenta por cento dos cargos em comissão devem ser \nprovidos por servidor público de carreira, nos casos e condições previstos em lei. \n§ 3º É proibid a a designação para função de confiança ou a nomeação para \ncargo em comissão, incluídos os de natureza especial, de pessoa que tenha praticado \nato tipificado como causa de inelegibilidade prevista na legislação eleitoral, observado o \nmesmo prazo de incompatibilidade dessa legislação. \nArt. 6º  As funções de confiança, privativas de servidor efetivo, destinam -se \nexclusivamente às atribuições de direção, chefia e assessoramento. \nArt. 7º São requisitos básicos para investidura em cargo público:  \nI – a nacionalidade brasileira; \nII – o gozo dos direitos políticos

## Load and Process PDF Research Papers with Contextual Information

In [ ]:
# create chunk context generation chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

def generate_chunk_context(document, chunk):

    chunk_process_prompt = """
    Você é um assistente de IA especializado na análise de documentos jurídicos.
    Sua tarefa é fornecer um contexto breve e relevante para um trecho de texto com base no seguinte documento jurídico.

    Aqui está o documento jurídico:
    <paper>
    {paper}
    </paper>

    Aqui está o trecho que queremos situar dentro do documento completo:
    <chunk>
    {chunk}
    </chunk>

    Forneça um contexto conciso (máximo de 3 a 4 frases) para esse trecho, seguindo as diretrizes abaixo:

    Dê um contexto curto e direto para situar esse trecho dentro do documento, visando melhorar a recuperação de busca do trecho.
    Responda apenas com o contexto conciso, sem adicionar informações extras.
    O contexto deve ser apresentado como "Foca em..." e não como "Este trecho ou seção foca em...".
    Contexto:
  """
    prompt_template = ChatPromptTemplate.from_template(chunk_process_prompt)
    agentic_chunk_chain = (prompt_template
                                |
                               chatgpt
                                |
                            StrOutputParser())
    context = agentic_chunk_chain.invoke({'paper': document, 'chunk': chunk})
    return context

In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid
from langchain.schema import Document

def create_contextual_chunks(file_path, chunk_size=3500, chunk_overlap=0):
    print('Loading pages:', file_path)
    loader = PyMuPDFLoader(file_path)
    doc_pages = loader.load()
    print('Chunking pages:', file_path)
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                              chunk_overlap=chunk_overlap)
    doc_chunks = splitter.split_documents(doc_pages)
    print('Generating contextual chunks:', file_path)
    original_doc = '\n'.join([doc.page_content for doc in doc_chunks])
    contextual_chunks = []
    for chunk in doc_chunks:
        chunk_content = chunk.page_content
        chunk_metadata = chunk.metadata
        chunk_metadata_upd = {
            'id': str(uuid.uuid4()),
            'page': chunk_metadata['page'],
            'source': chunk_metadata['source'],
            'title': chunk_metadata['source'].split('/')[-1]
        }
        context = generate_chunk_context(original_doc, chunk_content)
        contextual_chunks.append(Document(page_content=context+'\n'+chunk_content,
                                          metadata=chunk_metadata_upd))
    print('Finished processing:', file_path)
    print()
    return contextual_chunks

In [ ]:
import os
print(os.listdir('/content/drive/MyDrive/RAG'))

['Lei-Complementar-8402011.pdf', '.ipynb_checkpoints', 'paper_docs.pkl', 'paper_docs.json']


In [ ]:
from glob import glob

pdf_files = glob('/content/drive/MyDrive/RAG/*.pdf')
paper_docs = []
for fp in pdf_files:
    paper_docs.extend(create_contextual_chunks(file_path=fp,
                                               chunk_size=3500))

Loading pages: /content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf
Chunking pages: /content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf
Generating contextual chunks: /content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf


KeyboardInterrupt: 

In [ ]:
pdf_files[0]

'/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf'

In [ ]:
paper_docs[1]

Document(metadata={'id': 'cfe6dd18-f653-44d4-8f20-1552ab0ab454', 'page': 0, 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf', 'title': 'Lei-Complementar-8402011.pdf'}, page_content='Foca em definir as características e a natureza dos cargos em comissão dentro do regime jurídico dos servidores públicos civis do Distrito Federal. Especifica que esses cargos são de livre nomeação e exoneração pela autoridade competente, detalhando as categorias de direção, chefia e assessoramento.\ndireção, chefia e assessoramento, são de livre nomeação e exoneração pela autoridade \ncompetente. \n§ 1º Para os fins desta Lei Complementar, considera-se cargo em comissão: \nI – de direção: aquele cujo desempenho envolva atribuições da administração \nsuperior; \nII – de chefia: aquele cujo desempenho envolva relação direta e imediata de \nsubordinação; \nIII – de assessoramento: aquele cujas atribuições sejam para auxiliar:')

In [ ]:
import pickle

# Save to a pickle file
with open('/content/drive/MyDrive/RAG/paper_docs.pkl', 'wb') as f:
    pickle.dump(paper_docs, f)

In [ ]:
import pickle
# Load back when needed
with open('/content/drive/MyDrive/RAG/paper_docs.pkl', 'rb') as f:
    loaded_paper_docs = pickle.load(f)

In [ ]:
loaded_paper_docs[1]

Document(metadata={'id': 'cfe6dd18-f653-44d4-8f20-1552ab0ab454', 'page': 0, 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf', 'title': 'Lei-Complementar-8402011.pdf'}, page_content='Foca em definir as características e a natureza dos cargos em comissão dentro do regime jurídico dos servidores públicos civis do Distrito Federal. Especifica que esses cargos são de livre nomeação e exoneração pela autoridade competente, detalhando as categorias de direção, chefia e assessoramento.\ndireção, chefia e assessoramento, são de livre nomeação e exoneração pela autoridade \ncompetente. \n§ 1º Para os fins desta Lei Complementar, considera-se cargo em comissão: \nI – de direção: aquele cujo desempenho envolva atribuições da administração \nsuperior; \nII – de chefia: aquele cujo desempenho envolva relação direta e imediata de \nsubordinação; \nIII – de assessoramento: aquele cujas atribuições sejam para auxiliar:')

In [ ]:
total_docs = docs + loaded_paper_docs
len(total_docs)

218

## Create Vector Database Index and Setup Semantic Retrieval

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')
# create vector DB of docs and embeddings - takes < 30s on Colab - LEVOU 7 SEGUNDOS
chroma_db = Chroma.from_documents(documents=total_docs,
                                  collection_name='my_context_db',
                                  embedding=openai_embed_model,
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="/content/drive/MyDrive/my_context_db")

In [ ]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 5})

## Create BM25 Index and Setup Keyword Retrieval

In [ ]:
from langchain.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents=total_docs,k=5)

## Enable Hybrid Search with Ensemble Retrieval

In [ ]:
from langchain.retrievers import EnsembleRetriever
# reciprocal rank fusion
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, similarity_retriever],
    weights=[0.5, 0.5]
)

## Improving Retriever with Reranker

In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 86.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever

# download an open-source reranker model - BAAI/bge-reranker-v2-m3
reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
reranker_compressor = CrossEncoderReranker(model=reranker, top_n=5)
# Retriever 2 - Uses a Reranker model to rerank retrieval results from the previous retriever
final_retriever = ContextualCompressionRetriever(
    base_compressor=reranker_compressor,
    base_retriever=ensemble_retriever
)

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

## Testing our Retrieval Pipeline

In [ ]:
from IPython.display import display, Markdown
def display_docs(docs):
    for doc in docs:
        print('Metadata:', doc.metadata)
        print('Content Brief:')
        display(Markdown(doc.page_content[:1000]))
        print()
query = "O que é cargo público?"
top_docs = final_retriever.invoke(query)
display_docs(top_docs)

Metadata: {'page': 1, 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf'}
Content Brief:


 
a) os detentores de mandato eletivo; 
b) os ocupantes de cargos vitalícios; 
c) os ocupantes de cargos de direção ou de chefia. 
§ 2º Pelo menos cinquenta por cento dos cargos em comissão devem ser 
providos por servidor público de carreira, nos casos e condições previstos em lei. 
§ 3º É proibid a a designação para função de confiança ou a nomeação para 
cargo em comissão, incluídos os de natureza especial, de pessoa que tenha praticado 
ato tipificado como causa de inelegibilidade prevista na legislação eleitoral, observado o 
mesmo prazo de incompatibilidade dessa legislação. 
Art. 6º  As funções de confiança, privativas de servidor efetivo, destinam -se 
exclusivamente às atribuições de direção, chefia e assessoramento. 
Art. 7º São requisitos básicos para investidura em cargo público:  
I – a nacionalidade brasileira; 
II – o gozo dos direitos políticos; 
III – a quitação com as obrigações militares e eleitorais; 
IV – o nível de escolaridade exigido para o exercício do cargo; 



Metadata: {'id': 'cfe6dd18-f653-44d4-8f20-1552ab0ab454', 'page': 0, 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf', 'title': 'Lei-Complementar-8402011.pdf'}
Content Brief:


Foca em definir as características e a natureza dos cargos em comissão dentro do regime jurídico dos servidores públicos civis do Distrito Federal. Especifica que esses cargos são de livre nomeação e exoneração pela autoridade competente, detalhando as categorias de direção, chefia e assessoramento.
direção, chefia e assessoramento, são de livre nomeação e exoneração pela autoridade 
competente. 
§ 1º Para os fins desta Lei Complementar, considera-se cargo em comissão: 
I – de direção: aquele cujo desempenho envolva atribuições da administração 
superior; 
II – de chefia: aquele cujo desempenho envolva relação direta e imediata de 
subordinação; 
III – de assessoramento: aquele cujas atribuições sejam para auxiliar:


Metadata: {'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf', 'page': 7}
Content Brief:


 
I – julgar, em única e última instância, qualquer recurso interposto na forma do 
art. 29; 
II – homologar o resultado da avaliação especial feita pela comissão e, como 
consequência, efetivar o servidor no cargo, quando ele for aprovado no estágio 
probatório. 
Art. 31. O servidor reprovado no estágio probatório deve ser, conforme o caso, 
exonerado ou reconduzido ao cargo de origem. 
Seção VI 
Da Estabilidade 
Art. 32.  O servidor ocupante de cargo de provimento efetivo regularmente 
aprovado no estágio probatório adquire estabilidade no serviço público ao completar 
três anos de efetivo exercício. 
Art. 33.  O servidor estável só perde o cargo nas hipóteses previstas na 
Constituição Federal. 
Seção VII 
Da Reversão 
Art. 34. Reversão é o retorno à atividade de servidor aposentado: 
I – por invalidez, quando, por junta médica oficial, ficar comprovada a su a 
reabilitação; 
II – quando constatada, administrativa ou judicialmente, a insubsistência dos 
fundamentos de concessão da a


Metadata: {'id': 'cc994bd7-997a-4759-8731-0f89c3baeb34', 'page': 19, 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf', 'title': 'Lei-Complementar-8402011.pdf'}
Content Brief:


Foca em disposições sobre vantagens periódicas dos servidores públicos civis do Distrito Federal, especificamente o adicional de férias e o décimo terceiro salário. Estabelece os critérios de cálculo e pagamento dessas vantagens, assegurando direitos financeiros aos servidores durante suas férias e ao final do ano.
Seção VII 
Das Vantagens Periódicas 
Subseção I 
Do Adicional de Férias 
Art. 91. Independentemente de solicitação, é pago ao servidor, por ocasião 
das férias, um adicional correspondente a um terço da remuneração ou subsídio do mês 
em que as férias forem iniciadas.  
§ 1º No caso de o servidor efetivo exercer função de confiança ou cargo em 
comissão, a respectiva vantagem é considerada no cálculo do adicional de que trata 
este artigo, observada a proporcionalidade de que trata o art. 121, § 1º.  
§ 2º O adicional de férias incide sobre o valor do abono pecuniário.  
§ 3º A base para o cálculo do adicional de férias não pode ser superior ao teto 
de remuneração ou subsíd


Metadata: {'id': '162cd60c-a957-4d57-afea-a74f9ce0b8d8', 'page': 18, 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf', 'title': 'Lei-Complementar-8402011.pdf'}
Content Brief:


Foca em disposições sobre adicionais e gratificações que os servidores públicos do Distrito Federal podem receber, incluindo o adicional por serviço extraordinário e o adicional noturno, além de vantagens pessoais e adicionais por tempo de serviço e qualificação.
cinco, dez ou vinte por cento, na forma do regulamento.  
§ 2º A gratificação por trabalhos com raios X ou substâncias radioativas é 
concedida no percentual de dez por cento.  
Subseção III 
Do Adicional por Serviço Extraordinário 
Art. 84. O serviço extraordinário é remunerado com acréscimo de cinquenta 
por cento em relação ao valor da remuneração ou subsídio da hora normal de trabalho.  
Subseção IV 
Do Adicional Noturno 
Art. 85. O serviço noturno a que se refere o art. 59 é remunerado com 
acréscimo de vinte e cinco por cento sobre o valor da remuneração ou subsídio da hora 
trabalhada.  
Parágrafo único. O adicional noturno incide sobre o adicional de serviço 
extraordinário.  
Seção VI 
Das Vantagens Pessoais 
Subseção

In [ ]:
query = "Como servidora pública do GDF, faço jus a quais tipos de licença?"
top_docs = final_retriever.invoke(query)
display_docs(top_docs)

Metadata: {'id': 'b1e80935-0a82-433e-8e3a-2361103276c6', 'page': 28, 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf', 'title': 'Lei-Complementar-8402011.pdf'}
Content Brief:


Foca em disposições sobre a indenização de férias não gozadas e a regulamentação das licenças a que os servidores públicos têm direito, incluindo motivos como afastamento do cônjuge, doença em pessoa da família, serviço militar, atividade política, entre outros. O trecho estabelece critérios para a indenização das férias e detalha as condições para a concessão de diferentes tipos de licença.
ou aposentadoria, as férias não gozadas são indenizadas pelo valor da remuneração ou 
subsídio devido no mês da ocorrência do evento, acrescido do adicional de férias.  
§ 1º O período de férias incompleto é indenizado na proporção de um doze 
avos por mês de efetivo exercício.  
§ 2º Para os efeitos do § 1º, a fração superior a quatorze dias é considerada 
como mês integral.  
CAPÍTULO III 
DAS LICENÇAS 
Seção I 
Das Disposições Gerais 
Art. 130. Além do abono de ponto, o servidor faz jus a licença:  
I – por motivo de afastamento do cônjuge ou companheiro; 
II – por motivo de doença em pessoa da 


Metadata: {'id': '5cec4bc9-b6ee-41f6-aebf-3886f769cefa', 'page': 29, 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf', 'title': 'Lei-Complementar-8402011.pdf'}
Content Brief:


Foca em disposições sobre licenças concedidas aos servidores públicos do Distrito Federal, especificamente a licença para o serviço militar e a licença para atividade política. O artigo 136 trata da licença para servidores convocados ao serviço militar, enquanto o artigo 137 aborda as condições para a licença durante campanhas eleitorais.
Seção IV 
Da Licença para o Serviço Militar 
Art. 136. Ao servidor convocado para o serviço militar é concedida licença, na 
forma e nas condições previstas na legislação específica.  
Parágrafo único. Concluído o serviço militar, o servidor tem até trinta dias sem 
remuneração para reassumir o exercício do cargo.  
Seção V 
Da Licença para Atividade Política 
Art. 137. O servidor tem direito a licença para atividade política nos períodos 
compreendidos entre:  
I – a data de sua escolha em convenção partidária como candidato a cargo 
eletivo e a véspera do registro da candidatura perante a Justiça Eleitoral;  
II – o registro da candidatura perante a


Metadata: {'id': '1bd9e55f-0d6d-4a37-ac61-0e59c4bb4bc9', 'page': 31, 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf', 'title': 'Lei-Complementar-8402011.pdf'}
Content Brief:


Foca em regulamentar as licenças dos servidores públicos do Distrito Federal, especificamente a licença para o desempenho de mandato classista em entidades sindicais. Estabelece condições para a concessão, interrupção e prorrogação da licença, além de assegurar que a remuneração e encargos sociais sejam mantidos durante o período de licença.
II – não se encontre respondendo a processo disciplinar.  
§ 1º A licença pode ser interrompida, a qualquer tempo, a pedido do servidor 
ou a critério da administração.  
§ 2º O servidor não pode exercer cargo ou emprego público inacumulável 
durante a licença de que trata este artigo.  
§ 3º A licença pode ser prorrogada por igual período, uma única vez.  
Seção VIII 
Da Licença para o Desempenho de Mandato Classista 
Art. 145. Fica assegurado ao servidor estável o direito a licença para o 
desempenho de mandato em central sindical, confederação, federação ou sindicato 
representativos de servidores do Distrito Federal, regularmente registrados no


Metadata: {'id': 'd13b6dd5-0536-45f3-8a56-bbde8f03606c', 'page': 37, 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf', 'title': 'Lei-Complementar-8402011.pdf'}
Content Brief:


Foca em disposições sobre a contagem do tempo de serviço dos servidores públicos civis do Distrito Federal, especificando quais períodos não são considerados para esse cálculo, como faltas injustificadas e licenças sem remuneração. Além disso, aborda a definição de efetivo exercício e os períodos que não contam para essa consideração.
pelo qual o servidor receba proventos.  
Art. 164. Salvo disposição legal em contrário, não são contados como tempo 
de serviço:  
I – a falta injustificada ao serviço e a não compensada na forma desta Lei 
Complementar;  
II – o período em que o servidor estiver:  
a) licenciado ou afastado sem remuneração;  
b) cumprindo sanção disciplinar de suspensão; 
III – o período decorrido entre:  
a) a exoneração e o exercício em outro cargo de provimento efetivo;  
b) a concessão de aposentadoria voluntária e a reversão; 
c) a data de publicação do ato de reversão, reintegração, recondução ou 
aproveitamento e o retorno ao exercício do cargo.  
Art. 165. São co


Metadata: {'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf', 'page': 11}
Content Brief:


 
autarquia ou fundação onde o processo foi instaurado;  
II – provada a má-fé, aplicar a sanção de demissão, destituição ou cassação de 
aposentadoria ou disponibilidade em relação aos cargos ou empregos em  regime de 
acumulação ilegal, hipótese em que os órgãos ou entidades de vinculação devem ser 
comunicados.  
Art. 49. É vedada a participação de servidor, salvo na condição de Secretário 
de Estado, ainda que suplente, em mais de um conselho, comissão, comitê , órgão de 
deliberação coletiva ou assemelhado, na administração direta, autárquica ou 
fundacional do Distrito Federal.  
§ 1º É vedada a remuneração pela participação em mais de um conselho.  
§ 2º É permitida, observado o disposto no § 1º, a participação r emunerada de 
servidor em conselho de administração ou conselho fiscal de empresa pública ou 
sociedade de economia mista em que o Distrito Federal detenha, direta ou 
indiretamente, participação no capital social.  
CAPÍTULO V 
DA VACÂNCIA 
Art. 50. A vacância do cargo

##Building our Contextual RAG Pipeline

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """Você é um assistente altamente especializado em responder perguntas com base em informações fornecidas.
                Utilize exclusivamente o contexto recuperado para formular sua resposta.

                - Se a resposta não estiver no contexto, não tente adivinhar ou inventar, apenas informe que não possui essa informação.
                - Estruture sua resposta de forma clara, objetiva e bem formatada, mantendo o máximo de detalhes relevantes.
                - Responda em Português.
                - Cite os artigos em que as respostas foram baseadas.

                Pergunta:
                {question}

                Contexto:
                {context}

                Resposta:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = ({
    "context": (final_retriever
                |
                format_docs),
    "question": RunnablePassthrough()
    }
                |
                rag_prompt_template
                |
                chatgpt)

In [ ]:
from IPython.display import display, Markdown
query = "Como servidora pública do GDF, faço jus a quais tipos de licença?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Como servidora pública do GDF, você tem direito a diversas licenças, conforme estabelecido no Art. 130. As licenças a que você pode fazer jus incluem:

1. **Licença por motivo de afastamento do cônjuge ou companheiro**.
2. **Licença por motivo de doença em pessoa da família**.
3. **Licença para o serviço militar**.
4. **Licença para atividade política**.
5. **Licença prêmio por assiduidade**.
6. **Licença para tratar de interesses particulares**.
7. **Licença para desempenho de mandato classista**.
8. **Licença de paternidade**.
9. **Licença de maternidade**.
10. **Licença médica ou odontológica**.

Essas licenças são regulamentadas por normas específicas, e a concessão de cada uma delas pode estar sujeita a condições e prazos determinados pela legislação. Para mais detalhes sobre cada tipo de licença, consulte os artigos mencionados, especialmente o Art. 130 e os subsequentes que tratam das especificidades de algumas licenças, como a licença para o serviço militar (Art. 136) e a licença para atividade política (Art. 137).

In [ ]:
query = "O que é um cargo público?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Um cargo público é uma posição ocupada por um servidor que exerce funções em uma entidade pública, sendo regido por normas específicas que definem sua investidura, provimento e requisitos. De acordo com o contexto, os requisitos básicos para a investidura em um cargo público incluem:

1. Nacionalidade brasileira;
2. Gozo dos direitos políticos;
3. Quitação com as obrigações militares e eleitorais;
4. Nível de escolaridade exigido para o exercício do cargo;
5. Idade mínima de dezoito anos;
6. Aptidão física e mental.

Além disso, a lei pode estabelecer requisitos específicos para a investidura em cargos públicos, e os requisitos devem ser comprovados no momento da posse (Art. 7º).

As formas de provimento de cargo público incluem: nomeação, reversão, aproveitamento, reintegração e recondução (Art. 8º). O ato de provimento é de competência de autoridades específicas, como o Governador no Poder Executivo e o Presidente da Câmara Legislativa (Art. 10).

Os cargos públicos podem ser de diferentes naturezas, como os cargos em comissão, que são de livre nomeação e exoneração pela autoridade competente, e são destinados a funções de direção, chefia e assessoramento (Art. 6º e § 1º).

Essas informações estão baseadas nos artigos 6º, 7º, 8º e 10 do contexto fornecido.

In [ ]:
query = "Quando começa a contagem dos prazos prescricionais de uma ação disciplinar?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

A contagem dos prazos prescricionais de uma ação disciplinar começa a partir da primeira data em que o fato ou ato se tornou conhecido pela chefia da repartição onde ocorreu, pela chefia mediata ou imediata do servidor, ou pela autoridade competente para instaurar sindicância ou processo disciplinar. 

Essa informação está detalhada no § 1º do Art. 208. 

Além disso, a prescrição pode ser interrompida pela instauração de processo disciplinar, conforme o § 2º do mesmo artigo, e a contagem é reiniciada após a conclusão do processo disciplinar, conforme o § 3º. 

Referências:
- Art. 208, § 1º, § 2º e § 3º.

## Evaluation Metrics

In [ ]:
memory.clear()

In [ ]:
!pip install datasets
from datasets import Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
questions = ["Qual é o percentual aplicado pela Lei Complementar 840/2011 para reserva de vagas às pessoas portadoras de deficiência?",
             "A realização de concurso público é obrigatório para que?",
             "Quais fatores são observados durante o estágio probatório de um servidor público?",
             "Tenho direito a folga para consulta ou exames de rotina?",
             "Para fazer mestrado ou doutorado, tenho direito a horário diferenciado ou licença?",
             "Fui aprovado em concurso público federal, tendo direito a licença remunerada?",
             "O que acontece se eu acumular dois cargos públicos?",
             "Estou próximo a me aposentar, tenho alguma restrição?",
             "Posso ter um outro trabalho na esfera privada?",
             "Como é a licença prêmio/servidor?"
            ]

ground_truths = [["O percentual aplicado para reserva de vagas às pessoas portadoras de deficiência é de 20%"],
                ["A realização de concurso público é obrigatório para provimento de cargo efetivo e emprego público"],
                ["São observadas a responsabilidade, a assiduidade, a capacidade de iniciativa e a produditividades."],
                ["Sem prejuízo da remuneração ou subsídio, o servidor pode ausentar-se do serviço, mediante comunicação prévia à chefia imediata: por um dia para realizar, uma vez por ano, exames médicos preventivos ou periódicos voltados ao controle de câncer de próstata, de mama ou do colo de útero"],
                ["Pode ser concedido horário especial ao servidor matriculado em curso da educação básica e da educação superior, quando comprovada a incompatibilidade entre o horário escolar e o da unidade administrativa, sem prejuízo do exercício do cargo. É exigida do servidor a compensação de horário na unidade administrativa, de modo a cumprir integralmente o regime semanal de trabalho. O servidor estudante deve comprovar, mensalmente, a sua frequência escolar. Sobre o afastamento para participar de programa de Pós-Graduação Strictu Sensu, o servidor estável pode, no interesse da administração pública, e desde que a partici­pação não possa ocorrer simultaneamente com o exercício do cargo ou mediante compensação horário, afastar-se do exercício do cargo efetivo, com a respectiva remuneração ou subsídio, para participar de programa de pós-graduação stricto sensu em instituição de ensino superior, no País ou no exterior. O afastamento para realização de programas de mestrado, doutorado ou pós-doutorado somente pode ser concedido ao servidor estável que esteja em efetivo exercício no respectivo órgão, autarquia ou fundação há pelo menos três anos consecutivos para mestrado e quatro anos consecutivos para doutorado ou pós-doutorado."],
                ["Pode gozar de afastamento caso haja expressa previsão do curso de formação no edital do concurso e incompatibilidade entre os horários das aulas e os da repartição. O afastamento só pode ser com remuneração ou subsídio, nos casos de curso de formação para cargo efetivo de órgão, autarquia ou fundação dos Poderes Legislativo ou Executivo do Distrito Federal."],
                ["A Lei Complementar nº 840/2011 permite a acumulação de cargos públicos apenas nas hipóteses previstas na Constituição Federal (art. 37, XVI), como:a) dois cargos de professor; b) um cargo de professor e outro técnico ou científico; c) dois cargos ou empregos privativos de profissionais de saúde. Se um servidor acumular cargos de forma irregular, ele pode ser punido com demissão. O processo ocorre por meio de um PAD, que assegura o direito ao contraditório e à ampla defesa antes da aplicação da penalidade."],
                ["Algumas possíveis restrições: a) é necessário que o servidor tenha completado o tempo de contribuição e a idade exigidos por lei para se aposentar voluntariamente; b) apenas servidores efetivos podem solicitar a aposentadoria voluntária, c) se um servidor estiver respondendo a um Processo Administrativo Disciplinar (PAD) que possa resultar em demissão, seu pedido de aposentadoria poderá ser negado até a conclusão do processo; d) caso o servidor tenha sofrido penalidades graves, como demissão por infração disciplinar, pode perder o direito à aposentadoria estatutária; e) se o servidor tiver se ausentado por afastamento para estudo ou missão no exterior ou para participar de programa de Pós-Graduação Stricto Sensu e aposentar antes de decorrido período igual ao do afastamento, o servidor de ressarcir proporcionalmente a despesa, incluída a remuneração ou o subsídio e encargos sociais."],
                ["A Lei Complementar nº 840/2011 não proíbe o exercício de atividade remunerada no setor privado, desde que não haja conflito de interesses com o cargo público e que a atividade não comprometa a jornada de trabalho do servidor. São consideradas infrações: exercer atividade privada incompatível com o horário do serviço; exercer atividade privada incompatível com o exercício do cargo público ou da função de confiança; participar de gerência ou administração de sociedade ou empresa privada, personificada ou não personificada, salvo: a) nos casos previstos nesta Lei Complementar; b) nos períodos de licença ou afastamento do cargo sem remuneração, desde que não haja proi­bição em sentido contrário, nem incompatibilidade; c) em instituições ou entidades beneficentes, filantrópicas, de caráter social e humanitário e sem fins lucrativos, quando compatíveis com a jornada de trabalho."],
                ["A licença prêmio foi substituída pela licença servidor pela Lei Complementar 952/2019. Após cada quinquênio de efetivo exercício, o servidor ocupante de cargo efetivo faz jus a 3 meses de licença-servidor, sem prejuízo de sua remuneração, inclusive da retribuição do cargo em comissão, função de confiança ou função gratificada escolar - FGE que eventualmente exerça. Os períodos não são acumuláveis e não podem ser convertidos em pecúnia. O número de servidores afastados em virtude de licença-servidor não pode ser superior a 1/3 da lotação da respectiva unidade administrativa do órgão, autarquia ou fundação. A administração tem o prazo de até 120 dias, contado da data de requerimento do pedido pelo servidor, para definir o período de gozo da licença. Fica assegurado às servidoras e aos servidores o direito de iniciar a fruição de licença-servidor logo após o término da licença-maternidade ou da licença-paternidade."]]

answers = []
contexts = []
references = []

# Inference
# Inference
for query in questions:
  # Extract the generated text from the LLM response
  response = qa_rag_chain.invoke(query)
  # Access content using .content attribute
  answers.append(response.content)
  contexts.append([docs.page_content for docs in final_retriever.invoke(query)])
  # Add the ground truth as the reference here
  ground_truth_for_query = ground_truths[questions.index(query)]
  references.append(ground_truth_for_query)

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    # **Remove 'ground_truths', keep 'reference' as a string**
    "reference": [item[0] for item in ground_truths]  # Extract the string from the list
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [ ]:
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.3/178.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.8 MB/s eta 0:00:00


In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,Qual é o percentual aplicado pela Lei Compleme...,[ \nArt. 11. As normas gerais sobre concurso ...,O percentual aplicado pela Lei Complementar 84...,O percentual aplicado para reserva de vagas às...,1.000000,1.000000,1.000000,0.985412
1,A realização de concurso público é obrigatório...,[Foca em disposições relacionadas ao concurso ...,A realização de concurso público é obrigatória...,A realização de concurso público é obrigatório...,0.887500,1.000000,1.000000,0.856819
2,Quais fatores são observados durante o estágio...,[Foca em disposições relacionadas ao estágio p...,Durante o estágio probatório de um servidor pú...,"São observadas a responsabilidade, a assiduida...",0.916667,1.000000,0.538462,1.000000
3,Tenho direito a folga para consulta ou exames ...,[Foca em disposições sobre a jornada de trabal...,"Sim, você tem direito a folga para consultas o...","Sem prejuízo da remuneração ou subsídio, o ser...",1.000000,1.000000,0.666667,0.994958
4,"Para fazer mestrado ou doutorado, tenho direit...",[ \n§ 3º A jornada de trabalho em sistema de e...,Para servidores que desejam fazer mestrado ou ...,Pode ser concedido horário especial ao servido...,1.000000,0.600000,0.714286,0.000000
5,"Fui aprovado em concurso público federal, tend...",[Foca em disposições sobre o tempo de serviço ...,"Com base no contexto fornecido, não há informa...",Pode gozar de afastamento caso haja expressa p...,1.000000,1.000000,1.000000,0.000000
6,O que acontece se eu acumular dois cargos públ...,[Foca em disposições sobre a acumulação de car...,A acumulação de dois cargos públicos é regulad...,A Lei Complementar nº 840/2011 permite a acumu...,1.000000,0.666667,0.785714,0.915424
7,"Estou próximo a me aposentar, tenho alguma res...",[Foca em disposições sobre a aposentadoria por...,"Com base no contexto fornecido, não há informa...",Algumas possíveis restrições: a) é necessário ...,1.000000,0.400000,0.600000,0.000000
8,Posso ter um outro trabalho na esfera privada?,"[ \nconteúdo privado, informações e outros con...","Com base no contexto fornecido, a possibilidad...",A Lei Complementar nº 840/2011 não proíbe o ex...,1.000000,1.000000,1.000000,0.000000
9,Como é a licença prêmio/servidor?,[Foca em disposições relacionadas à licença-pr...,A licença-prêmio para servidores públicos é um...,A licença prêmio foi substituída pela licença ...,0.866667,0.000000,0.846154,0.927644


In [ ]:
df.to_excel("dados_Contextual_RAG_openAI.xlsx", index=False)